### do imports


In [1]:
from sqlalchemy.engine import create_engine
from sqlalchemy import text
from pydantic import BaseModel

### setup connection to MySQL database


In [2]:
# define MySQL connection details

mysql_username = "shield_admin"
mysql_password = "shield_4dmin"

mysql_endpoint = "shield-1.c3uu4ke88lpy.eu-west-3.rds.amazonaws.com"
mysql_port = 3306

mysql_database = "users_db"

connection_url = "mysql+pymysql://{username}:{password}@{endpoint}:{port}/{database}".\
    format(username=mysql_username,
           password=mysql_password,
           endpoint=mysql_endpoint,
           port=mysql_port,
           database=mysql_database)

In [3]:
# create MySQL engine

mysql_engine = create_engine(connection_url)

In [4]:
# view main_table columns

with mysql_engine.connect() as connection:
    query = "SHOW COLUMNS FROM main_table;"
    results = connection.execute(text(query))

results.fetchall()

[('id', 'varchar(16)', 'NO', 'PRI', None, ''),
 ('username', 'varchar(16)', 'NO', '', None, ''),
 ('password', 'varchar(16)', 'NO', '', None, ''),
 ('rights', 'tinyint(1)', 'NO', '', None, '')]

In [5]:
# view main_table contents

with mysql_engine.connect() as connection:
    query = "SELECT * FROM main_table;"
    results = connection.execute(text(query))

results.fetchall()

[('admin', 'admin', '4dmin', 1),
 ('fdo', 'fdo', 'c0ps', 0),
 ('policierA', 'policierA', 'sherif', 0),
 ('policierB', 'policierB', 'colombo', 0)]

### implement MySQL in `post_user()`

(coroutine for EP2 `/register`)


In [6]:
# define sample arguments

class User(BaseModel):
    username: str
    password: str
    rights: int


new_user = User(username="alice",
                password="wonderland",
                rights=0)

identification="admin:4dmin"

In [7]:
# simulate credential processing

user, psw = identification.split(":")

In [8]:
with mysql_engine.connect() as connection:

    # get authorization test parameter

    query = "SELECT rights FROM main_table WHERE username = '{}';".format(user)
    results = connection.execute(text(query))
    rights = results.scalar()

    # get authentication test parameter

    query = "SELECT password FROM main_table WHERE username = '{}';".format(user)
    results = connection.execute(text(query))
    password = results.scalar()

In [9]:
# simulate successful authorization

if rights == 1:

    # simulate successful authentication

    if password == psw:

        # add new_user to main_table

        with mysql_engine.connect() as connection:
            query = "INSERT INTO main_table (id, username, password, rights)\
                    VALUES ('{}', '{}', '{}', {});".format(new_user.username,
                                                           new_user.username,
                                                           new_user.password,
                                                           new_user.rights)
            connection.execute(text(query))
            connection.commit()

In [10]:
# view updated main_table

with mysql_engine.connect() as connection:
    query = "SELECT * FROM main_table"
    results = connection.execute(text(query))

results.fetchall()

[('admin', 'admin', '4dmin', 1),
 ('alice', 'alice', 'wonderland', 0),
 ('fdo', 'fdo', 'c0ps', 0),
 ('policierA', 'policierA', 'sherif', 0),
 ('policierB', 'policierB', 'colombo', 0)]